In [70]:
import backtrader as bt
import backtrader.analyzers as btanalyzers
import pandas as pd
import yfinance as yf
import requests
from bs4 import BeautifulSoup

import matplotlib
import matplotlib.pyplot as plt

from datetime import datetime

In [71]:
class YahooFinanceDataWithDividends(bt.feeds.PandasData):
    params = (
        ("dataname", None),
        ("fromdate", None),
        ("todate", None),
        ("dividends", True),
    )

In [72]:
class MaCrossStrategy(bt.Strategy):
    params = (
        ("fast_period", 10),
        ("slow_period", 50),
        ("fib_levels", [0, 0.236, 0.382, 0.618, 1.0]),  # Fibonacci levels
        ("risk_reward_ratio", 2),
    )

    def __init__(self):
        ma_fast = bt.ind.SMA(period=self.params.fast_period)
        ma_slow = bt.ind.SMA(period=self.params.slow_period)
        self.crossover = bt.ind.CrossOver(ma_fast, ma_slow)

    def next(self):
        if not self.position:
            if self.crossover > 0:
                self.buy()
        elif self.crossover < 0:
            self.close()

In [73]:
cerebro = bt.Cerebro()
stocks = ["AAPL"]
startdate = "2010-01-01"
enddate = "2020-01-01"

for stock_symbol in stocks:
    # Create a Ticker object for the stock
    stock = yf.Ticker(stock_symbol)

    # Get dividend data for the stock within the date range
    dividends = stock.dividends
    dividends = dividends[(dividends.index >= startdate) & (dividends.index <= enddate)]

    # Get stock split data for the stock within the date range
    splits = stock.splits
    splits = splits[(splits.index >= startdate) & (splits.index <= enddate)]

    # Print dividend and stock split data for the current stock
    print(f"Dividend Dates for {stock_symbol}:")
    print(dividends)

    print(f"\nStock Split Dates for {stock_symbol}:")
    print(splits)

Dividend Dates for AAPL:
Date
2012-08-09 00:00:00-04:00    0.094643
2012-11-07 00:00:00-05:00    0.094643
2013-02-07 00:00:00-05:00    0.094643
2013-05-09 00:00:00-04:00    0.108929
2013-08-08 00:00:00-04:00    0.108929
2013-11-06 00:00:00-05:00    0.108929
2014-02-06 00:00:00-05:00    0.108929
2014-05-08 00:00:00-04:00    0.117500
2014-08-07 00:00:00-04:00    0.117500
2014-11-06 00:00:00-05:00    0.117500
2015-02-05 00:00:00-05:00    0.117500
2015-05-07 00:00:00-04:00    0.130000
2015-08-06 00:00:00-04:00    0.130000
2015-11-05 00:00:00-05:00    0.130000
2016-02-04 00:00:00-05:00    0.130000
2016-05-05 00:00:00-04:00    0.142500
2016-08-04 00:00:00-04:00    0.142500
2016-11-03 00:00:00-04:00    0.142500
2017-02-09 00:00:00-05:00    0.142500
2017-05-11 00:00:00-04:00    0.157500
2017-08-10 00:00:00-04:00    0.157500
2017-11-10 00:00:00-05:00    0.157500
2018-02-09 00:00:00-05:00    0.157500
2018-05-11 00:00:00-04:00    0.182500
2018-08-10 00:00:00-04:00    0.182500
2018-11-08 00:00:00-

In [75]:
stock = yf.Ticker(stock_symbol)
data = stock.history(period="1h", start=startdate, end=enddate)
volume_per_hour = data['Volume'].resample('H').sum()

print("Trading Volume per Hour for the Last Two Weeks:")
print(volume_per_hour)
plt.figure(figsize=(12, 6))
plt.plot(volume_per_hour)
plt.title(f"Trading Volume per Hour for {stock_symbol} (Last Two Weeks)")
plt.xlabel("Date")
plt.ylabel("Volume")
plt.grid(True)
plt.show()

Trading Volume per Hour for the Last Two Weeks:
Date
2010-01-04 00:00:00-05:00    493729600
2010-01-04 01:00:00-05:00            0
2010-01-04 02:00:00-05:00            0
2010-01-04 03:00:00-05:00            0
2010-01-04 04:00:00-05:00            0
                               ...    
2019-12-30 20:00:00-05:00            0
2019-12-30 21:00:00-05:00            0
2019-12-30 22:00:00-05:00            0
2019-12-30 23:00:00-05:00            0
2019-12-31 00:00:00-05:00    100805600
Freq: H, Name: Volume, Length: 87553, dtype: int64


<IPython.core.display.Javascript object>

In [58]:
# Chart for stock split and dividends
plt.figure(figsize=(12, 6))
if not splits.empty:
    plt.plot(splits.index, splits, 'go', markersize=8, label="Stock Splits")
if not dividends.empty:
    plt.plot(dividends.index, dividends, 'ro', markersize=8, label="Dividends")

plt.title(f"Stock Splits and Dividends for {stock_symbol}")
plt.xlabel("Date")
plt.ylabel("Value")
plt.legend()
plt.grid(True)
plt.show()


<IPython.core.display.Javascript object>

In [59]:
# Yearly revenue 
data = stock.history(start=startdate, end=enddate)
yearly_revenue = data['Close'].resample('Y').sum()

plt.figure(figsize=(12, 6))
plt.plot(yearly_revenue.index, yearly_revenue, marker='o', linestyle='-')
plt.title(f"Yearly Revenue for {stock_symbol}")
plt.xlabel("Year")
plt.ylabel("Yearly Revenue")
plt.grid(True)
plt.show()


<IPython.core.display.Javascript object>

In [60]:

data = YahooFinanceDataWithDividends(
        dataname=yf.download("AAPL", start=startdate, end=enddate)
    )
stocks

cerebro.adddata(data)
cerebro.addstrategy(MaCrossStrategy)
cerebro.broker.setcash(1000000.0)
cerebro.addsizer(bt.sizers.PercentSizer, percents = 10)
# cerebro.addsizer(bt.sizers.SizerFix, stake=3)

# https://www.backtrader.com/docu/analyzers/analyzers/
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name = "sharpe")
cerebro.addanalyzer(btanalyzers.Transactions, _name = "trans")
cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name = "trades")

[*********************100%%**********************]  1 of 1 completed


In [61]:
back = cerebro.run()

In [62]:

cerebro.broker.getvalue()

1192215.1850040953

In [63]:
back[0].analyzers.sharpe.get_analysis()

OrderedDict([('sharperatio', 0.42264346093622995)])

In [64]:
back[0].analyzers.trans.get_analysis()

OrderedDict([(datetime.datetime(2010, 8, 2, 0, 0),
              [[10884.3537414966,
                9.30142879486084,
                0,
                '',
                -101240.04130460779]]),
             (datetime.datetime(2010, 8, 17, 0, 0),
              [[-10884.3537414966,
                8.931428909301758,
                0,
                '',
                97212.83166586948]]),
             (datetime.datetime(2010, 9, 14, 0, 0),
              [[10443.094080616525,
                9.507499694824219,
                0,
                '',
                -99287.71378448221]]),
             (datetime.datetime(2011, 3, 21, 0, 0),
              [[-10443.094080616525,
                11.999643325805664,
                0,
                '',
                125313.40418523073]]),
             (datetime.datetime(2011, 5, 3, 0, 0),
              [[8263.820572211142,
                12.428214073181152,
                0,
                '',
                -102704.53113379843]])

In [65]:
back[0].analyzers.trades.get_analysis()

AutoOrderedDict([('total',
                  AutoOrderedDict([('total', 30),
                                   ('open', 1),
                                   ('closed', 29)])),
                 ('streak',
                  AutoOrderedDict([('won',
                                    AutoOrderedDict([('current', 3),
                                                     ('longest', 3)])),
                                   ('lost',
                                    AutoOrderedDict([('current', 0),
                                                     ('longest', 4)]))])),
                 ('pnl',
                  AutoOrderedDict([('gross',
                                    AutoOrderedDict([('total',
                                                      147079.11511990288),
                                                     ('average',
                                                      5071.693624824237)])),
                                   ('net',
                            

In [66]:
cerebro.plot()

<IPython.core.display.Javascript object>

[[<Figure size 640x480 with 5 Axes>]]